In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
def init_spark():
    spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()
    return spark
spark_object = init_spark()

23/03/29 18:45:17 WARN Utils: Your hostname, karyubuntu resolves to a loopback address: 127.0.1.1; using 172.31.170.192 instead (on interface wlp0s20f3)
23/03/29 18:45:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/29 18:45:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
train_identity = spark_object.read.option("delimiter", ",").csv("./data/train_identity.csv", header='true', inferSchema='true')
train_transaction = spark_object.read.option("delimiter", ",").csv("./data/train_transaction.csv", header='true', inferSchema='true')

In [4]:
train = train_transaction.join(train_identity, on="TransactionID", how='inner')

In [5]:
total_count = train.count()
def null_percentage(df, column):
    null_count = df.where(col(column).isNull()).count()
    return null_count / total_count

column_list = [] 
for column in train.columns:
    if null_percentage(train, column) > 0.05:
        train = train.drop(col(column))
    else:
        column_list.append(column)

In [6]:
print((train.count(), len(train.columns)))
train.toPandas().to_csv('./data/traincleaned.csv' )

(144233, 194)
23/03/29 19:03:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


work with test data

In [ ]:
test_identity = spark_object.read.option("delimiter", ",").csv("./data/test_identity.csv", header='true', inferSchema='true')
test_transaction = spark_object.read.option("delimiter", ",").csv("./data/test_transaction.csv", header='true', inferSchema='true')

In [ ]:
test = test_transaction.join(test_identity, on="TransactionID", how='inner')
for name in test.columns:
    new_name = name.replace('-', '_')
    if (new_name != name):
        test = test.withColumnRenamed(name,new_name)

In [ ]:
list_column = (train.columns)
for column in test.columns:
    if column not in list_column:
        test = test.drop(col(column))

In [ ]:
print(len(train.columns))
print((test.count(), len(test.columns)))

194
(141907, 193)


In [ ]:
test.toPandas().to_csv('./data/testcleaned.csv')